# **Cuarto conjunto de tareas a realizar**

## Paquetes necesarios e inicializaciones

La siguiente práctica consta dos partes principales, la primera de ellas basada en YOLO y en detección de matrículas personas y vehículos y la segunda en OCR (Optical Character Recognition).

Instalar labelme en el sistema operativo del anfitrión (NO EN EL ENVIRONMENT)

Hacer lo siguiente desde el CMD del sistema para pasar las imágenes en formato JSON obtenidas con labelme y pasarlas a formato YOLO.

Si se tiene una tarjeta gráfica de NVIDIA se puede utilizar la GPU haciendo uso de CUDA, para instalar CUDAv11.6 hacer uso del siguiente script.

Realizar las importaciones de las librerías y paquetes necesarios

In [16]:
import cv2
import pandas as pd
import csv
import numpy as np
from ultralytics import YOLO
import easyocr
import os
from collections import defaultdict
import torch
import traceback
from transformers import AutoProcessor, AutoModelForCausalLM
from PIL import Image

Realizar el entrenamiento del modelo haciendo uso de yolo11l.pt (en caso de no tenerlo se descargará automáticamente al ejecutar el entrenamiento).

In [ ]:
# === ENTRENAMIENTO DEL MODELO ===
print("Iniciando entrenamiento del modelo")

model = YOLO("yolo11l.pt")

# Entrenamiento con el dataset de matrículaas personalizado
results = model.train(
    data="data.yaml",               # Ruta al archivo de configuración de datos
    imgsz=640,                      # Tamaño de las imágenes (Se reescalan para más eficiencia)
    epochs=300,                     # Número de épocas de entrenamiento
    project="runs/train_custom",    # Directorio donde se guardarán los resultados
    name="exp2",                    # Nombre del entrenamiento
    exist_ok=True,                  # Sobrescribir si el directorio ya existe
    plots=True                      # Generar gráficos de métricas durante el entrenamiento
)

print("Entrenamiento completado.")

Iniciando entrenamiento del modelo
Ultralytics 8.3.223  Python-3.9.23 torch-1.12.1 CUDA:0 (NVIDIA GeForce RTX 3060 Ti, 8192MiB)
WARNING Upgrade to torch>=2.0.0 for deterministic training.
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=300, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11l.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=exp2, nbs=64, nms=False, opset=None, optimize=False,

Comprobar el funcionamiento del modelo detector de matrículas junto al modelo por defecto de YOLO en donde se detectan personas y vehículos (elegido a mano).

Adicionalmente, se prueba el OCR encima de las matrículas y se guarda el vídeo en el directorio **[out](/VC_P4/out/)**.

In [ ]:
# ============================
# Directorio de Salida
# ============================
output_folder = "out"
os.makedirs(output_folder, exist_ok=True)

# ============================
# Modelos
# ============================
custom_model = YOLO("runs/train_custom/exp2/weights/best.pt")
coco_model = YOLO("yolo11l.pt")

vehicle_class_indices = [2, 3, 5, 7] # car, motorcycle, bus, truck
person_class_index = 0
coco_class_indices = [person_class_index] + vehicle_class_indices

print("Cargando modelo EasyOCR")
ocr_reader = easyocr.Reader(['es', 'en'], gpu=True) 
print("Modelo EasyOCR cargado.")

# ============================
# Funciones de Reconocimiento de Matrículas
# ============================

def recognize_plate_ocr(plate_image, reader):
    """Procesa la imagen de la matrícula usando EasyOCR."""
    try:
        results = reader.readtext(plate_image)
        text = " ".join([res[1] for res in results])
        clean = "".join(filter(str.isalnum, text)).upper()

        if clean == "":
            return "OCR_ERROR"

        return clean

    except Exception:
        return "OCR_ERROR"

# ============================
# Tracker simple
# ============================
object_id_counter = 0
prev_centroids = {}

def get_centroid(box):
    x1, y1, x2, y2 = map(int, box)
    return int((x1+x2)/2), int((y1+y2)/2)

def assign_id(cx, cy, prev_centroids, threshold=50):
    global object_id_counter
    for oid, (pcx, pcy) in prev_centroids.items():
        if np.sqrt((cx-pcx)**2 + (cy-pcy)**2) < threshold:
            prev_centroids[oid] = (cx, cy)
            return oid
    # Si no se encuentra, asignar nuevo ID
    object_id_counter += 1
    prev_centroids[object_id_counter] = (cx, cy)
    return object_id_counter

# ============================
# Dibujar bounding boxes
# ============================
def draw_boxes(results, frame, color=(0,255,0), filter_classes=None):
    for r in results:
        boxes = r.boxes
        for i, (box, cls, conf) in enumerate(zip(boxes.xyxy, boxes.cls, boxes.conf)):
            cls = int(cls)
            if filter_classes and cls not in filter_classes:
                continue
            x1, y1, x2, y2 = map(int, box)
            
            label = f"{r.names[cls]} {conf:.2f}"
            
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(frame, label, (x1, y1-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    return frame

# ============================
# Video
# ============================
video_path = "./Resources/video.mp4"
cap = cv2.VideoCapture(video_path)

# Configuración del VideoWriter
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

output_video_path = os.path.join(output_folder, "video_con_detecciones_ocr.mp4")
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out_writer = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

# ============================
# CSV
# ============================
csv_file = os.path.join(output_folder, "resultsOCR.csv")
csv_headers = [
    "fotograma", "tipo_objeto", "confianza", "id_tracking", 
    "x1", "y1", "x2", "y2",
    "matricula_en_su_caso", "conf_matricula", "mx1","my1","mx2","my2", 
    "texto_OCR"
]

f = open(csv_file, mode='w', newline='', encoding='utf-8')
writer = csv.writer(f)
writer.writerow(csv_headers)

# ============================
# Procesar TODOS los frames
# ============================
frame_num = 0
print("Iniciando procesamiento de todo el video...")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
        
    frame_num += 1

    if frame_num % 50 == 0:
        print(f"Procesando frame {frame_num}...")

    # Detecciones de COCO
    coco_results = coco_model.predict(frame, imgsz=640, conf=0.25, verbose=False)
    vehicle_boxes_current = []
    
    # Procesar resultados de COCO y Tracking
    for r in coco_results:
        boxes = r.boxes
        for box, cls, conf in zip(boxes.xyxy, boxes.cls, boxes.conf):
            cls = int(cls)
            if cls not in coco_class_indices:
                continue
            x1, y1, x2, y2 = map(int, box)
            cx, cy = get_centroid((x1, y1, x2, y2))
            oid = assign_id(cx, cy, prev_centroids)
            tipo_obj = r.names[cls]
            
            # Escribir la detección del vehículo/persona
            writer.writerow([frame_num, tipo_obj, float(conf), oid, x1, y1, x2, y2,
                              "", "", "", "", "", "", 
                              ""])

    # Detecciones MATRÍCULAS
    custom_results = custom_model.predict(frame, imgsz=640, conf=0.25, verbose=False) 

    for r in custom_results:
        boxes = r.boxes
        for box, cls, conf in zip(boxes.xyxy, boxes.cls, boxes.conf):
            x1, y1, x2, y2 = map(int, box)
            
            cx, cy = get_centroid((x1, y1, x2, y2))
            oid = assign_id(cx, cy, prev_centroids)
            
            plate_crop = frame[y1:y2, x1:x2]
            
            # Reconocimiento por OCR
            texto_ocr = recognize_plate_ocr(plate_crop, ocr_reader)
            
            # Escribir la detección de la matrícula y los resultados
            writer.writerow([
                frame_num, r.names[int(cls)], float(conf), oid, x1, y1, x2, y2,
                "plate", float(conf), x1, y1, x2, y2, 
                texto_ocr # Resultado OCR
            ])

            # Dibujar la BBox de la matrícula aquí
            label = f"{r.names[int(cls)]} {conf:.2f} | OCR: {texto_ocr}"
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
            cv2.putText(frame, label, (x1, y1-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

    # ============================
    # Dibujar bounding boxes COCO
    # ============================
    frame = draw_boxes(coco_results, frame, color=(0,255,0), filter_classes=coco_class_indices)
    
    # Escribir el frame en el video de salida
    out_writer.write(frame)

cap.release()
out_writer.release()
f.close()

print("\nProcesamiento completado.")
print(f"CSV guardado en {csv_file}")
print(f"Video guardado en {output_video_path}")

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


Cargando modelo EasyOCR
Modelo EasyOCR cargado.
Iniciando procesamiento de todo el video...
Procesando frame 50...
Procesando frame 100...
Procesando frame 150...
Procesando frame 200...
Procesando frame 250...
Procesando frame 300...
Procesando frame 350...
Procesando frame 400...
Procesando frame 450...
Procesando frame 500...
Procesando frame 550...
Procesando frame 600...
Procesando frame 650...
Procesando frame 700...
Procesando frame 750...


c:\Users\ivanp\anaconda3\envs\VC_P4\lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Procesando frame 800...
Procesando frame 850...
Procesando frame 900...
Procesando frame 950...
Procesando frame 1000...
Procesando frame 1050...
Procesando frame 1100...
Procesando frame 1150...
Procesando frame 1200...
Procesando frame 1250...
Procesando frame 1300...
Procesando frame 1350...
Procesando frame 1400...
Procesando frame 1450...
Procesando frame 1500...
Procesando frame 1550...
Procesando frame 1600...
Procesando frame 1650...
Procesando frame 1700...
Procesando frame 1750...
Procesando frame 1800...
Procesando frame 1850...
Procesando frame 1900...
Procesando frame 1950...
Procesando frame 2000...
Procesando frame 2050...
Procesando frame 2100...
Procesando frame 2150...
Procesando frame 2200...
Procesando frame 2250...
Procesando frame 2300...
Procesando frame 2350...
Procesando frame 2400...
Procesando frame 2450...
Procesando frame 2500...
Procesando frame 2550...
Procesando frame 2600...
Procesando frame 2650...
Procesando frame 2700...
Procesando frame 2750...
Proc

Comprobar el funcionamiento del modelo detector de matrículas junto al modelo por defecto de YOLO en donde se detectan personas y vehículos (elegido a mano).

Adicionalmente, se prueba el VML encima de las matrículas y se guarda el vídeo en el directorio **[out](/VC_P4/out/)**.

In [20]:
# ============================
# Directorio de Salida
# ============================
output_folder = "out"
os.makedirs(output_folder, exist_ok=True)

# ============================
# Modelos
# ============================
custom_model = YOLO("runs/train_custom/exp2/weights/best.pt")
coco_model = YOLO("yolo11l.pt")

vehicle_class_indices = [2, 3, 5, 7] # car, motorcycle, bus, truck
person_class_index = 0
coco_class_indices = [person_class_index] + vehicle_class_indices

### Inicializar VLM (Florence-2) ###
print("Cargando modelo VLM (Florence-2)...")
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Usando dispositivo: {device}")

model_id = 'microsoft/Florence-2-large'
model = AutoModelForCausalLM.from_pretrained(
    model_id, 
    trust_remote_code=True, 
    attn_implementation="eager"
).to(device)

processor = AutoProcessor.from_pretrained(model_id, trust_remote_code=True)
print("Modelo VLM cargado.")

# ============================
# Funciones de Reconocimiento de Matrículas
# ============================

def recognize_plate_vlm(plate_image, model, processor, device):
    """Procesa la imagen de la matrícula usando Florence-2 VLM."""
    try:
        image_rgb = cv2.cvtColor(plate_image, cv2.COLOR_BGR2RGB)
        pil_image = Image.fromarray(image_rgb)
        
        prompt = "<OCR>" # Tarea específica de OCR de Florence-2
        
        inputs = processor(text=prompt, images=pil_image, return_tensors="pt").to(device) # <-- ### CORRECCIÓN 2: .half() eliminado para CPU ###
        
        generated_ids = model.generate(
            input_ids=inputs["input_ids"],
            pixel_values=inputs["pixel_values"],
            max_new_tokens=50,
            do_sample=False,
            num_beams=3,
            use_cache=False
        )
        
        generated_text = processor.batch_decode(generated_ids, skip_special_tokens=False)[0]
        parsed_text = processor.post_process_generation(generated_text, task=prompt, image_size=(pil_image.width, pil_image.height))
        
        text_from_vlm = parsed_text.get('<OCR>', 'VLM_PARSE_ERROR')

        return "".join(filter(str.isalnum, text_from_vlm)).upper()
    
    except Exception as e:
        print(f"--- ERROR DETALLADO EN VLM (Inicio) ---")
        print(f"Tipo de Error: {type(e).__name__}")
        print(f"Mensaje: {e}")
        traceback.print_exc() 
        print(f"--- ERROR DETALLADO EN VLM (Fin) ---")
        return "VLM_ERROR"

# ============================
# Tracker simple
# ============================
object_id_counter = 0
prev_centroids = {}

def get_centroid(box):
    x1, y1, x2, y2 = map(int, box)
    return int((x1+x2)/2), int((y1+y2)/2)

def assign_id(cx, cy, prev_centroids, threshold=50):
    global object_id_counter
    for oid, (pcx, pcy) in prev_centroids.items():
        if np.sqrt((cx-pcx)**2 + (cy-pcy)**2) < threshold:
            prev_centroids[oid] = (cx, cy)
            return oid
    object_id_counter += 1
    prev_centroids[object_id_counter] = (cx, cy)
    return object_id_counter

# ============================
# Dibujar bounding boxes
# ============================
def draw_boxes(results, frame, color=(0,255,0), filter_classes=None):
    for r in results:
        boxes = r.boxes
        for i, (box, cls, conf) in enumerate(zip(boxes.xyxy, boxes.cls, boxes.conf)):
            cls = int(cls)
            if filter_classes and cls not in filter_classes:
                continue
            x1, y1, x2, y2 = map(int, box)
            
            label = f"{r.names[cls]} {conf:.2f}"
            
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(frame, label, (x1, y1-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    return frame

# ============================
# Video
# ============================
video_path = "./Resources/video.mp4"
cap = cv2.VideoCapture(video_path)

# Configuración del VideoWriter
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

output_video_path = os.path.join(output_folder, "video_con_detecciones_VLM.mp4")
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out_writer = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

# ============================
# CSV
# ============================
csv_file = os.path.join(output_folder, "resultsVLM.csv")
csv_headers = [
    "fotograma", "tipo_objeto", "confianza", "id_tracking", 
    "x1", "y1", "x2", "y2",
    "matricula_en_su_caso", "conf_matricula", "mx1","my1","mx2","my2", 
    "texto_VLM"
]

f = open(csv_file, mode='w', newline='', encoding='utf-8')
writer = csv.writer(f)
writer.writerow(csv_headers)

# ============================
# Procesar TODOS los frames
# ============================
frame_num = 0
print("Iniciando procesamiento de todo el video (con VLM)...")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
        
    frame_num += 1

    if frame_num % 10 == 0: 
        print(f"Procesando frame {frame_num}...")

    # Detecciones de COCO
    coco_results = coco_model.predict(frame, imgsz=640, conf=0.25, verbose=False)
    
    # Procesar resultados de COCO y Tracking
    for r in coco_results:
        boxes = r.boxes
        for box, cls, conf in zip(boxes.xyxy, boxes.cls, boxes.conf):
            cls = int(cls)
            if cls not in coco_class_indices:
                continue
            x1, y1, x2, y2 = map(int, box)
            cx, cy = get_centroid((x1, y1, x2, y2))
            oid = assign_id(cx, cy, prev_centroids)
            tipo_obj = r.names[cls]
            
            # Escribir la detección del vehículo/persona
            writer.writerow([frame_num, tipo_obj, float(conf), oid, x1, y1, x2, y2,
                              "", "", "", "", "", "", 
                              ""])

    # Detecciones MATRÍCULAS
    custom_results = custom_model.predict(frame, imgsz=640, conf=0.25, verbose=False) 

    for r in custom_results:
        boxes = r.boxes
        for box, cls, conf in zip(boxes.xyxy, boxes.cls, boxes.conf):
            x1, y1, x2, y2 = map(int, box)
            
            cx, cy = get_centroid((x1, y1, x2, y2))
            oid = assign_id(cx, cy, prev_centroids)
            
            plate_crop = frame[y1:y2, x1:x2]

            texto_vlm = recognize_plate_vlm(plate_crop, model, processor, device)
            
            writer.writerow([
                frame_num, r.names[int(cls)], float(conf), oid, x1, y1, x2, y2,
                "plate", float(conf), x1, y1, x2, y2, 
                texto_vlm
            ])

            label = f"{r.names[int(cls)]} {conf:.2f} | VLM: {texto_vlm}"
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
            cv2.putText(frame, label, (x1, y1-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

    # ============================
    # Dibujar bounding boxes COCO
    # ============================
    frame = draw_boxes(coco_results, frame, color=(0,255,0), filter_classes=coco_class_indices)
    
    # Escribir el frame en el video de salida
    out_writer.write(frame)

cap.release()
out_writer.release()
f.close()

print("\nProcesamiento completado.")
print(f"CSV guardado en {csv_file}")
print(f"Video guardado en {output_video_path}")

Cargando modelo VLM (Florence-2)...
Usando dispositivo: cpu
Modelo VLM cargado.
Iniciando procesamiento de todo el video (con VLM)...
Procesando frame 10...
Procesando frame 20...
Procesando frame 30...
Procesando frame 40...
Procesando frame 50...
Procesando frame 60...
Procesando frame 70...
Procesando frame 80...
Procesando frame 90...
Procesando frame 100...
Procesando frame 110...
Procesando frame 120...
Procesando frame 130...
Procesando frame 140...
Procesando frame 150...
Procesando frame 160...
Procesando frame 170...
Procesando frame 180...
Procesando frame 190...
Procesando frame 200...
Procesando frame 210...
Procesando frame 220...
Procesando frame 230...
Procesando frame 240...
Procesando frame 250...
Procesando frame 260...
Procesando frame 270...
Procesando frame 280...
Procesando frame 290...
Procesando frame 300...
Procesando frame 310...
Procesando frame 320...
Procesando frame 330...
Procesando frame 340...
Procesando frame 350...
Procesando frame 360...
Procesando 

Realizar diferentes pruebas sobre imágenes estáticas para comprobar el funcionamiento del OCR de manera más exacta.

In [ ]:
input_folder = "C:/Users/ivanp/Desktop/CarPlates/TGC_RBNW/train/images/"
output_folder = "out/images_OCR"

os.makedirs(output_folder, exist_ok=True) 

print("Cargando modelo EasyOCR")
ocr_reader = easyocr.Reader(['es', 'en'], gpu=True) 
print("Modelo EasyOCR cargado.")

images_iter = 8

csv_output_path = os.path.join(output_folder, "ocr_results.csv")
csv_file = open(csv_output_path, mode='w', newline='', encoding='utf-8')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(["filename", "ground_truth", "ocr_result", "is_correct"])

try:
    all_files = os.listdir(input_folder)
    image_files = [f for f in all_files if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp'))]
    print(f"Se encontraron {len(image_files)} imágenes. Procesando las primeras {images_iter}.")
except FileNotFoundError:
    print(f"Error: No se encontró el directorio de entrada: {input_folder}")
    csv_file.close() # Cerramos el archivo si hay error
    exit()

for i, filename in enumerate(image_files[:images_iter]):
    image_path = os.path.join(input_folder, filename)

    ground_truth, ext = os.path.splitext(filename)
    ground_truth = ground_truth.upper()

    image = cv2.imread(image_path)

    if image is None:
        print(f"Error: No se pudo cargar la imagen en la ruta: {image_path}")
        csv_writer.writerow([filename, ground_truth, "IMAGE_LOAD_ERROR", False])
        continue
    
    results = ocr_reader.readtext(image)
    print(f"\n--- RESULTADOS OCR PARA {filename} ---")
    
    if not results:
        print("No se detectó texto.")
        csv_writer.writerow([filename, ground_truth, "NO_TEXT_DETECTED", False])
    else:
        all_text = []
        print("--- Texto Detectado ---")
        
        for (bbox, text, prob) in results:
            print(f'Texto: "{text}", Confianza: {prob:.4f}')
            
            if prob > 0.85: 
                all_text.append(text)
                
                # --- DIBUJAR EN LA IMAGEN ---
                tl = tuple(map(int, bbox[0])) # Convertir a enteros
                br = tuple(map(int, bbox[2])) # Convertir a enteros
                
                cv2.rectangle(image, tl, br, (0, 255, 0), 2)
                cv2.putText(image, text, (tl[0], tl[1] - 10), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
        
        print("\n--- Texto Limpio (formato matrícula) ---")
        full_text_joined = " ".join(all_text)
        clean_text = "".join(filter(str.isalnum, full_text_joined)).upper()
        print(clean_text)

        is_correct = (clean_text == ground_truth)
        csv_writer.writerow([filename, ground_truth, clean_text, is_correct])
        print(f"Acierto: {is_correct}")

        output_filename = f"{ground_truth}_processed{ext}"
        output_path = os.path.join(output_folder, output_filename)

        cv2.imwrite(output_path, image)
        print(f"Imagen procesada guardada en: {output_path}")

csv_file.close()
print("\n--- Procesamiento de las imágenes completado. ---")
print(f"Resultados de aciertos guardados en: {csv_output_path}")

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


Cargando modelo EasyOCR
Modelo EasyOCR cargado.
Se encontraron 122 imágenes. Procesando las primeras 8.

--- RESULTADOS OCR PARA 0116GPD.jpg ---
--- Texto Detectado ---
Texto: "Bayabor", Confianza: 0.0498
Texto: "0116 GPD", Confianza: 0.9960
Texto: "207", Confianza: 0.8104
Texto: "(KmZERO", Confianza: 0.8443

--- Texto Limpio (formato matrícula) ---
0116GPD
Acierto: True
Imagen procesada guardada en: out/images_OCR\0116GPD_processed.jpg

--- RESULTADOS OCR PARA 0116HGV.jpg ---
--- Texto Detectado ---
Texto: "0116 '", Confianza: 0.9659
Texto: "HGV", Confianza: 0.9219

--- Texto Limpio (formato matrícula) ---
0116HGV
Acierto: True
Imagen procesada guardada en: out/images_OCR\0116HGV_processed.jpg

--- RESULTADOS OCR PARA 0259MTF_6290KBG.jpg ---
--- Texto Detectado ---
Texto: "DL", Confianza: 0.9686
Texto: "P", Confianza: 0.9984
Texto: "Bec2522", Confianza: 0.0454
Texto: "0259 MTF", Confianza: 0.9589
Texto: "1", Confianza: 0.4891

--- Texto Limpio (formato matrícula) ---
DLP0259MTF
Aciert

Realizar diferentes pruebas sobre imágenes estáticas para comprobar el funcionamiento del VLM de manera más exacta.

In [ ]:

input_folder = "C:/Users/ivanp/Desktop/CarPlates/TGC_RBNW/train/images/"
output_folder = "out/images_VLM" 

os.makedirs(output_folder, exist_ok=True) 

# ============================
# Cargar Modelo VLM (Florence-2)
# ============================
print("Cargando modelo VLM (Florence-2)...")
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Usando dispositivo: {device}")

model_id = 'microsoft/Florence-2-large'
model = AutoModelForCausalLM.from_pretrained(
    model_id, 
    trust_remote_code=True, 
    attn_implementation="eager" 
).to(device)
processor = AutoProcessor.from_pretrained(model_id, trust_remote_code=True)
print("Modelo VLM cargado.")

def recognize_plate_vlm_with_boxes(image_bgr, model, processor, device):
    """Procesa la imagen y devuelve texto y bounding boxes usando VLM."""
    try:
        image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
        pil_image = Image.fromarray(image_rgb)
        
        prompt = "<OCR_WITH_REGION>" 
        inputs = processor(text=prompt, images=pil_image, return_tensors="pt").to(device) # <-- SIN .half()
        
        generated_ids = model.generate(
            input_ids=inputs["input_ids"],
            pixel_values=inputs["pixel_values"],
            max_new_tokens=1024,
            do_sample=False,
            num_beams=3,
            use_cache=False
        )
        
        generated_text = processor.batch_decode(generated_ids, skip_special_tokens=False)[0]
        parsed_text = processor.post_process_generation(
            generated_text, 
            task=prompt, 
            image_size=(pil_image.width, pil_image.height)
        )
        
        return parsed_text.get('<OCR_WITH_REGION>', {})
    except Exception as e:
        print(f"--- ERROR DETALLADO EN VLM (Inicio) ---")
        print(f"Tipo de Error: {type(e).__name__}")
        print(f"Mensaje: {e}")
        traceback.print_exc() 
        print(f"--- ERROR DETALLADO EN VLM (Fin) ---")
        return {}

images_iter = 8
csv_output_path = os.path.join(output_folder, "vlm_results.csv")
csv_file = open(csv_output_path, mode='w', newline='', encoding='utf-8')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(["filename", "ground_truth", "vlm_result_all_text", "is_correct"])

try:
    all_files = os.listdir(input_folder)
    image_files = [f for f in all_files if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp'))]
    print(f"Se encontraron {len(image_files)} imágenes. Procesando las primeras {images_iter} con VLM.")
except FileNotFoundError:
    print(f"Error: No se encontró el directorio de entrada: {input_folder}")
    csv_file.close() 
    exit()

for i, filename in enumerate(image_files[:images_iter]):
    image_path = os.path.join(input_folder, filename)
    print(f"\n--- Procesando imagen {i+1}/{images_iter}: {filename} ---")

    ground_truth, ext = os.path.splitext(filename)
    ground_truth = ground_truth.upper()

    image = cv2.imread(image_path)

    if image is None:
        print(f"Error: No se pudo cargar la imagen en la ruta: {image_path}")
        csv_writer.writerow([filename, ground_truth, "IMAGE_LOAD_ERROR", False])
        continue
    
    results = recognize_plate_vlm_with_boxes(image, model, processor, device)
    
    print(f"--- RESULTADOS VLM PARA {filename} ---")
    
    if not results:
        print("No se detectó texto.")
        csv_writer.writerow([filename, ground_truth, "NO_TEXT_DETECTED", False])
    else:
        all_text_for_csv = []
        is_correct = False
        
        print("--- Texto Detectado ---")

        if isinstance(results, dict) and 'labels' in results and 'quad_boxes' in results:
            
            print("DEBUG: VLM devolvió DICCIONARIO con 'labels' y 'quad_boxes'.")
            
            for text, box in zip(results['labels'], results['quad_boxes']):
                
                clean_label = text.replace('</s>', '').strip()
                clean_label = "".join(filter(str.isalnum, clean_label)).upper()

                if not clean_label:
                    continue
                    
                all_text_for_csv.append(clean_label)

                # Comparamos esta etiqueta limpia con la matrícula real
                if clean_label == ground_truth:
                    is_correct = True
                    print(f"¡Acierto encontrado!: {clean_label}")

                try:
                    points = list(map(int, box))
                    tl = (points[0], points[1]) # Top-left (x1, y1)
                    br = (points[4], points[5]) # Bottom-right (x3, y3)
                    
                    label_to_draw = text.replace('</s>', '').strip()
                    
                    print(f'Texto: "{label_to_draw}", Coordenadas (simple): {tl}, {br}')
                    
                    cv2.rectangle(image, tl, br, (0, 0, 255), 2)
                    cv2.putText(image, label_to_draw, (tl[0], tl[1] - 10), 
                                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
                                
                except Exception as e_draw:
                    print(f"Error al dibujar caja para '{label_to_draw}': {e_draw}")

        else:
            print(f"DEBUG: VLM devolvió un formato inesperado: {results}")
            csv_writer.writerow([filename, ground_truth, "UNEXPECTED_FORMAT", False])
            continue 

        full_clean_text = "".join(all_text_for_csv) 
        
        print("\n--- Texto Limpio (para CSV) ---")
        print(full_clean_text)

        csv_writer.writerow([filename, ground_truth, full_clean_text, is_correct])
        print(f"Acierto: {is_correct}")

        output_filename = f"{ground_truth}_processed{ext}"
        output_path = os.path.join(output_folder, output_filename)

        cv2.imwrite(output_path, image)
        print(f"Imagen procesada guardada en: {output_path}")

csv_file.close()
print("\n--- Procesamiento de las imágenes completado. ---")
print(f"Resultados de aciertos guardados en: {csv_output_path}")

Cargando modelo VLM (Florence-2)...
Usando dispositivo: cpu
Modelo VLM cargado.
Se encontraron 122 imágenes. Procesando las primeras 8 con VLM.

--- Procesando imagen 1/8: 0116GPD.jpg ---
--- RESULTADOS VLM PARA 0116GPD.jpg ---
--- Texto Detectado ---
DEBUG: VLM devolvió DICCIONARIO con 'labels' y 'quad_boxes'.
¡Acierto encontrado!: 0116GPD
Texto: "0116 GPD", Coordenadas (simple): (334, 789), (521, 848)
Texto: "KINZERO", Coordenadas (simple): (230, 805), (277, 828)
Texto: "HONDAI", Coordenadas (simple): (239, 824), (283, 842)
Texto: "207", Coordenadas (simple): (567, 846), (614, 865)

--- Texto Limpio (para CSV) ---
0116GPDKINZEROHONDAI207
Acierto: True
Imagen procesada guardada en: out/images_VLM\0116GPD_processed.jpg

--- Procesando imagen 2/8: 0116HGV.jpg ---
--- RESULTADOS VLM PARA 0116HGV.jpg ---
--- Texto Detectado ---
DEBUG: VLM devolvió DICCIONARIO con 'labels' y 'quad_boxes'.
¡Acierto encontrado!: 0116HGV
Texto: "0116 HGV", Coordenadas (simple): (350, 1092), (470, 1164)

--- T

Comparaciones generales entre el OCR y el VLM haciendo uso de los datos obtenidos mediante las imágenes estáticas de los códigos anteriores.

In [ ]:
ocr_csv_path = 'out/images_OCR/ocr_results.csv'
vlm_csv_path = 'out/images_VLM/vlm_results.csv'

if not os.path.exists(ocr_csv_path):
    print(f"Error: No se encontró el archivo {ocr_csv_path}")
    exit()
if not os.path.exists(vlm_csv_path):
    print(f"Error: No se encontró el archivo {vlm_csv_path}")
    exit()

try:
    df_ocr = pd.read_csv(ocr_csv_path)
    df_ocr = df_ocr.rename(columns={
        'ocr_result': 'texto_ocr',
        'is_correct': 'acierto_ocr'
    })
    df_ocr = df_ocr[['filename', 'ground_truth', 'texto_ocr', 'acierto_ocr']]

    df_vlm = pd.read_csv(vlm_csv_path)
    df_vlm = df_vlm.rename(columns={
        'vlm_result_all_text': 'texto_vlm',
        'is_correct': 'acierto_vlm'
    })
    df_vlm = df_vlm[['filename', 'texto_vlm', 'acierto_vlm']]
    
except Exception as e:
    print(f"Error al leer los CSV: {e}")
    exit()

# Unir (merge) las dos tablas usando 'filename' como la clave
try:
    df_comparacion = pd.merge(df_ocr, df_vlm, on='filename')
except Exception as e:
    print(f"Error al unir los CSV. ¿Tienen la columna 'filename'?: {e}")
    exit()

if df_comparacion.empty:
    print("No se encontraron imágenes en común entre los dos CSV.")
    exit()

total_imagenes = len(df_comparacion)

aciertos_ocr = df_comparacion['acierto_ocr'].sum()
aciertos_vlm = df_comparacion['acierto_vlm'].sum()

porcentaje_ocr = (aciertos_ocr / total_imagenes) * 100
porcentaje_vlm = (aciertos_vlm / total_imagenes) * 100

ambos_acertaron = len(df_comparacion[
    (df_comparacion['acierto_ocr'] == True) & 
    (df_comparacion['acierto_vlm'] == True)
])
ambos_fallaron = len(df_comparacion[
    (df_comparacion['acierto_ocr'] == False) & 
    (df_comparacion['acierto_vlm'] == False)
])
solo_ocr_gano = len(df_comparacion[
    (df_comparacion['acierto_ocr'] == True) & 
    (df_comparacion['acierto_vlm'] == False)
])
solo_vlm_gano = len(df_comparacion[
    (df_comparacion['acierto_ocr'] == False) & 
    (df_comparacion['acierto_vlm'] == True)
])

print("--- Reporte de Comparación EasyOCR vs. VLM ---")
print(f"Total de imágenes comparadas: {total_imagenes}")

print("\n--- Porcentaje de Acierto General ---")
print(f"EasyOCR: {porcentaje_ocr:.2f}% ({aciertos_ocr}/{total_imagenes})")
print(f"VLM:     {porcentaje_vlm:.2f}% ({aciertos_vlm}/{total_imagenes})")

print("\n--- Análisis Detallado (Head-to-Head) ---")
print(f"Imágenes donde AMBOS acertaron: {ambos_acertaron}")
print(f"Imágenes donde AMBOS fallaron:  {ambos_fallaron}")
print("---------------------------------------------")
print(f"Imágenes donde SOLO EasyOCR acertó: {solo_ocr_gano}")
print(f"Imágenes where SOLO VLM acertó:     {solo_vlm_gano}")

if solo_vlm_gano > 0:
    print("\n--- Ejemplos donde VLM ganó (OCR falló, VLM acertó) ---")
    print(df_comparacion[
        (df_comparacion['acierto_ocr'] == False) & 
        (df_comparacion['acierto_vlm'] == True)
    ][['filename', 'ground_truth', 'texto_ocr', 'texto_vlm']].to_string())

if solo_ocr_gano > 0:
    print("\n--- Ejemplos donde EasyOCR ganó (VLM falló, OCR acertó) ---")
    print(df_comparacion[
        (df_comparacion['acierto_ocr'] == True) & 
        (df_comparacion['acierto_vlm'] == False)
    ][['filename', 'ground_truth', 'texto_ocr', 'texto_vlm']].to_string())

--- Reporte de Comparación EasyOCR vs. VLM ---
Total de imágenes comparadas: 8

--- Porcentaje de Acierto General ---
EasyOCR: 25.00% (2/8)
VLM:     87.50% (7/8)

--- Análisis Detallado (Head-to-Head) ---
Imágenes donde AMBOS acertaron: 2
Imágenes donde AMBOS fallaron:  1
---------------------------------------------
Imágenes donde SÓLO EasyOCR acertó: 0
Imágenes where SÓLO VLM acertó:     5

--- Ejemplos donde VLM ganó (OCR falló, VLM acertó) ---
       filename ground_truth texto_ocr     texto_vlm
3   0290KWT.jpg      0290KWT      0290       0290KWT
4   0303BML.jpg      0303BML   BML0303       0303BML
5  0415JVS.jpeg      0415JVS   JVS0415       0415JVS
6   0416MLX.jpg      0416MLX       NaN       0416MLX
7  0463HVT.jpeg      0463HVT  0463HVTE  FIAT0463HVTE
